## 🔰PyTorchでニューラルネットワーク基礎 #24 【マルチラベル・文章分類】

### 内容
* Qiitaの記事と連動しています
* 簡易のBERTタイプでマルチラベル分類、Transformer Encoderを利用したマルチラベル分類となります
* data/MultiLabelDataSample.json 


### データについて
[chABSA-dataset](https://github.com/chakki-works/chABSA-dataset)を加工したデータを利用します。
* ラベル数：３（否定的、中立的、肯定的）をマルチラベルで扱う
* データ数：3300
* 単語数：6950
* 系列長（文章の長さ）：58

**いろいろ試した結果の一つ**
* Exact Match: 0.7782
* Hamming: 0.9079
* Macro F1: 0.7270

In [1]:
import json
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [2]:
#デバイスの選択
device = "cuda" if torch.cuda.is_available() else "cpu"
print("利用デバイス:", device)


def label_prediction(y, threshold=0.5):
    probs = torch.sigmoid(y)
    predictions = (probs >= threshold).float()
    return predictions


# 精度の計算 (Exact Match)
def accuracy(y,t, threshold=0.5):
    prediction = label_prediction(y, threshold=threshold)
    num_correct = (prediction == t).all(-1).sum().item()
    accuracy = num_correct/t.size(0)
    return accuracy

利用デバイス: cuda


In [3]:
# JSONファイルを読み込む
with open("./data/MultiLabelDataSample.json", "r") as f:
    data = json.load(f)

In [4]:
# リストに変換してからtensorに
# x: IDベクトル
# t: ラベルだけど、BCE損失を使うので、FloatにFloatにしておく
x = torch.LongTensor([item["ids"] for item in data])
t = torch.FloatTensor([item["labels"] for item in data])

num_positives = t.sum(dim=0)
num_negatives = len(t) - num_positives
# negative/positiveの比率をpos_weightに
pos_weight = torch.FloatTensor(num_negatives / num_positives)

x, x_test, t, t_test = train_test_split(x,t, stratify=t,  random_state=55)

x = x.to(device)
t = t.to(device)
x_test = x_test.to(device)
t_test = t_test.to(device)
pos_weight = pos_weight.to(device)
pos_weight, x.shape, x_test.shape

(tensor([ 2.2195, 23.6269,  1.4070], device='cuda:0'),
 torch.Size([2475, 58]),
 torch.Size([825, 58]))

In [5]:
# 初期設定（今回から数値を外側に出してみました）
WORDS = 6950    # 単語数 len(word2id)
SEQ_LEN = 58    # x.shape[1]  # 入力するIDベクトルの長さ
D_MODEL = 64    # 分散表現の次元
N_HEAD = 8      # マルチヘッドのヘッド数
DIM_FEEDFORWARD = 256 # TransformerEncoder層の中間層の特徴量の次元
CLASSES = 3     # t.shape[1] ラベルの数

In [6]:
class DNN(nn.Module):
    def __init__(
        self,
        pad_token_id: int=3,  # <pad> idの指定：デフォルト値0とあえて変更しているよ
    ):
        super().__init__()
        self.pad_token_id = pad_token_id
       
        # (1) トークン埋め込み
        self.token_embedding = nn.Embedding(num_embeddings=WORDS, embedding_dim=D_MODEL,padding_idx=self.pad_token_id)
        
        # (2) 学習可能な位置埋め込み（0〜max_len-1）
        self.pos_embedding = nn.Embedding(num_embeddings=SEQ_LEN, embedding_dim=D_MODEL)

        # (3) Layer Normalization と Dropout
        self.layer_norm = nn.LayerNorm(D_MODEL)
        self.dropout = nn.Dropout(0.1)
        
        # (4) Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=D_MODEL,
            nhead=N_HEAD,
            dim_feedforward=DIM_FEEDFORWARD,
            dropout=0.1,
            batch_first=True,   # [batch, seq, d_model] で扱えるように
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer,num_layers=6)
        
        # (5) 文ベクトル → クラス数
        self.classifier = nn.Linear(in_features=D_MODEL, out_features=CLASSES)
    
    def forward(self, x):
        # (6) マスクの変換 TransformerEncoder 用: PAD ID=3でPAD のところが True になる mask
        src_key_padding_mask = (x == self.pad_token_id)
        # (7) 埋め込み
        # トークン埋め込み
        tok_emb = self.token_embedding(x)  # [batch, seq_len=58, d_model=64]            
        # 位置埋め込み（broadcastingで自動拡張）
        pos_emb = self.pos_embedding(torch.arange(SEQ_LEN, device=x.device))  # [seq_len=58, d_model=64]
        # トークン埋め込み + 位置埋め込み                     
        x = tok_emb + pos_emb.unsqueeze(0)     # [batch, seq_len=58, d_model=64]
        
        # (8) Layer Normalization と Dropout
        x = self.layer_norm(x)
        x = self.dropout(x)
        
        # (9) Transformer Encoder
        h = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)
        
        # (10) 文ベクトルへの Pooling (PADを除外した平均)
        mask = (~src_key_padding_mask).unsqueeze(-1).float()  # [batch, seq_len, 1]
        pooled = (h * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1)  # [batch, d_model]
               
        # (11) 文ベクトルへの Pooling  <BOS>トークン（先頭）に情報を集約
        #pooled = h[:, 0, :]  # [batch, d_model]
        
        # (12) 分類
        y = self.classifier(pooled)  # [batch, num_labels=3]    
        return y

In [7]:
model = DNN()
model.to(device)

DNN(
  (token_embedding): Embedding(6950, 64, padding_idx=3)
  (pos_embedding): Embedding(58, 64)
  (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (classifier): Linear(in_features=64, out_features=3, bias=True)
)

In [8]:
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)

In [9]:
LOOP = 300

for epoch in range(LOOP):
    optimizer.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    acc = accuracy(y,t)
    
    loss.backward()
    optimizer.step()

    if (epoch+1)%30==0:  
        print(f"{epoch+1}:\tloss:{loss.item():.3f}\tacc:{acc:.3f}")


30:	loss:0.715	acc:0.415
60:	loss:0.548	acc:0.536
90:	loss:0.410	acc:0.657
120:	loss:0.321	acc:0.750
150:	loss:0.246	acc:0.789
180:	loss:0.225	acc:0.840
210:	loss:0.173	acc:0.861
240:	loss:0.129	acc:0.908
270:	loss:0.106	acc:0.922
300:	loss:0.097	acc:0.933


In [17]:
model.eval()
with torch.inference_mode():
    y_test = model(x_test)
acc = accuracy(y_test, t_test)
print(f"検証精度: {acc:.4f}")

検証精度: 0.7782


これ以降はわりと適当な感じです

In [ ]:
@torch.no_grad()
def evaluate_multilabel(y, t, threshold=0.5, eps=1e-10):
    """
    y: logits, shape [N, L]
    t: targets(0/1), shape [N, L]
    threshold: float もしくは 長さ3のリスト
    """
    probs = torch.sigmoid(y)

    # threshold を [1, L] に整形してブロードキャストできるようにする
    if isinstance(threshold, (float, int)):
        thr = torch.full((1, probs.size(1)), float(threshold), device=probs.device, dtype=probs.dtype)
    else:
        thr = torch.as_tensor(threshold, device=probs.device, dtype=probs.dtype).view(1, -1)
        if thr.size(1) != probs.size(1):
            raise ValueError(f"threshold length ({thr.size(1)}) must match num_labels ({probs.size(1)})")

    predictions = (probs >= thr).float()

    # Exact Match
    exact_acc = (predictions == t).all(dim=-1).float().mean().item()

    # Hamming Accuracy
    hamming_acc = (predictions == t).float().mean().item()

    # label-wise counts
    tp = (predictions * t).sum(dim=0)
    fp = (predictions * (1 - t)).sum(dim=0)
    fn = ((1 - predictions) * t).sum(dim=0)

    precision = tp / (tp + fp + eps)
    recall = tp / (tp + fn + eps)
    f1_per_label = 2 * precision * recall / (precision + recall + eps)

    macro_f1 = f1_per_label.mean().item()

    return {
        "exact_match": exact_acc,
        "hamming": hamming_acc,
        "macro_f1": macro_f1,
        "f1_per_label": f1_per_label.detach().cpu(), 
        "precision_per_label": precision.detach().cpu(),
        "recall_per_label": recall.detach().cpu(),
    }


In [19]:
threshold=[0.5, 0.5, 0.5]
metrics=evaluate_multilabel(y_test, t_test, threshold=threshold)

print("全体の指標")
print(f"Exact Match: {metrics['exact_match']:.4f}")
print(f"Hamming: {metrics['hamming']:.4f}")
print(f"Macro F1: {metrics['macro_f1']:.4f}")
print("\nラベルごとの指標")
print(f"f1 per label: {metrics['f1_per_label']}")
print(f"precision per label: {metrics['precision_per_label']}")
print(f"recall per label: {metrics['recall_per_label']}")

全体の指標
Exact Match: 0.7782
Hamming: 0.9079
Macro F1: 0.7270

ラベルごとの指標
f1 per label: tensor([0.8291, 0.5075, 0.8444])
precision per label: tensor([0.8307, 0.5000, 0.8348])
recall per label: tensor([0.8275, 0.5152, 0.8542])
